In [ ]:
from chebGreen import ChebGreen
import numpy as np

%env TF_MLC_LOGGING=1
%env TF_CPP_MIN_LOG_LEVEL=2

In [ ]:
airy = ChebGreen([1,5,10], generateData = True, script = "generate_example", example = "airy_equation")

In [ ]:
airy.G